In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
# Charger les 3 feuilles du fichier Excel dans 3 DataFrames différents
df1 = pd.read_excel("/home/onyxia/work/Statapp/Bases/parc_vp_region_2024_1.xlsx", sheet_name="Parc Régionaux", engine="openpyxl", header=3)
df2 = pd.read_excel("/home/onyxia/work/Statapp/Bases/parc_vp_region_2024_1.xlsx", sheet_name="Distance", engine="openpyxl", header=4)
df3 = pd.read_excel("/home/onyxia/work/Statapp/Bases/parc_vp_region_2024_1.xlsx", sheet_name="Parcours annuel moyen", engine="openpyxl", header=4)

# Propager les valeurs non-NaN dans les colonnes VP, Unnamed: 1 et Unnamed: 2
df2['VP'] = df2['VP'].ffill()
df2['Unnamed: 1'] = df2['Unnamed: 1'].ffill()
df2['Unnamed: 2'] = df2['Unnamed: 2'].ffill()
df3['VP'] = df3['VP'].ffill()
df3['Unnamed: 1'] = df3['Unnamed: 1'].ffill()
df3['Unnamed: 2'] = df3['Unnamed: 2'].ffill()


df1.to_csv("/home/onyxia/work/Statapp/Bases/parc_vp_region_2024_1_Dis.xlsx", index=False)
df2.to_csv("/home/onyxia/work/Statapp/Bases/parc_vp_region_2024_1_PAM.xlsx", index=False)
df3.to_csv("/home/onyxia/work/Statapp/Bases/parc_vp_region_2024_1_PR.xlsx", index=False)

In [13]:
# Afficher les premières lignes de chaque DataFrame pour vérifier
'''print("DF1:")
print(df1.head())
'''

print("\nDF2:")
print(df2.head(15))

'''
print("\nDF3:")
print(df3.head(15))'''


DF2:
       VP      Unnamed: 1                         Unnamed: 2          2011  \
0   Total           Total                              Total  455457190140   
1   Total           Total            Electrique et hydrogène      17863798   
2   Total           Total                  Essence thermique  113475302499   
3   Total           Total  Essence hybride non\nrechargeable     215339226   
4   Total           Total      Essence hybride\nrechargeable     375689738   
5   Total           Total                   Diesel thermique  337769329148   
6   Total           Total   Diesel hybride non\nrechargeable       4581368   
7   Total           Total       Diesel hybride\nrechargeable       1781506   
8   Total           Total                                Gaz    2725968535   
9   Total           Total                      Non déterminé     871334322   
10  Total  professionnels                              Total   48477184329   
11  Total  professionnels            Electrique et hydrogè

'\nprint("\nDF3:")\nprint(df3.head(15))'

In [17]:
# Extraire les lignes de 0 à 29
lignes_to_copy = df2.iloc[0:30]

# Insérer ces lignes après la ligne 29, donc dans les lignes 30 à 59
df_with_copied_rows = pd.concat([df2.iloc[:30], lignes_to_copy, df2.iloc[30:]]).reset_index(drop=True)

# Extraire les lignes de 30 à 59
df_lines_30_59 = df_with_copied_rows.iloc[30:60]

# Remplacer "Total" par "Moyennes régionales" dans la colonne "VP" (colonne 0) pour les lignes 30 à 59
df_lines_30_59.loc[:, 'VP'] = df_lines_30_59['VP'].replace('Total', 'Moyennes régionales')

# Diviser les valeurs des colonnes à partir de la 4ème colonne (incluse) par 20
df_lines_30_59 = df_lines_30_59.copy()
df_lines_30_59.loc[:, df_lines_30_59.columns[3:]] = df_lines_30_59.iloc[:, 3:] / 20



# Réinsérer ces lignes modifiées dans le DataFrame
df_with_modifications = pd.concat([df_with_copied_rows.iloc[:30], df_lines_30_59, df_with_copied_rows.iloc[60:]]).reset_index(drop=True)

#Afficher les premières lignes pour vérifier
print(df_with_copied_rows.iloc[25:60]) 

df_with_modifications.to_csv("/home/onyxia/work/Statapp/Bases/parc_vp_region_2024_1_Dis_V2.xlsx", index=False)

                     VP      Unnamed: 1                         Unnamed: 2  \
25                Total    particuliers                   Diesel thermique   
26                Total    particuliers   Diesel hybride non\nrechargeable   
27                Total    particuliers       Diesel hybride\nrechargeable   
28                Total    particuliers                                Gaz   
29                Total    particuliers                      Non déterminé   
30  Moyennes régionales           Total                              Total   
31  Moyennes régionales           Total            Electrique et hydrogène   
32  Moyennes régionales           Total                  Essence thermique   
33  Moyennes régionales           Total  Essence hybride non\nrechargeable   
34  Moyennes régionales           Total      Essence hybride\nrechargeable   
35  Moyennes régionales           Total                   Diesel thermique   
36  Moyennes régionales           Total   Diesel hybride non\nre

In [ ]:

'''
Tracer les distances parcourues par an selon les années selon les carburants. 
D'abord en total puis séparés selon professionels ou particuliers

'''
# Charger les données dans un DataFrame
df_distance_tot = df_with_modifications.iloc[:10].reset_index(drop=True)

# Renommer les colonnes pour éviter les "Unnamed" (à changer un peu parce que c'est pas sur que ce soit le bon nom)
df_distance_tot.rename(columns={"Unnamed: 1": "Statut", "Unnamed: 2": "Carburant"}, inplace=True)

# Filtrer pour exclure la ligne "Total" dans la colonne "Carburant"
df_distance_tot = df_distance_tot[df_distance_tot["Carburant"] != "Total"]

# Sélectionner les années (colonnes à partir de la 4ᵉ colonne)
years = df_distance_tot.columns[3:]


# Tracer les courbes
plt.figure(figsize=(12, 6))

for _, row in df_distance_tot.iterrows():
    plt.plot(years, row[3:].values, label=row["Carburant"])  # Tracer chaque ligne

# Personnalisation du graphique
plt.xlabel("Année")
plt.ylabel("Distance parcourue")
plt.title("Distance parcourue par type de carburant selon les années (particuliers et professionels)")
plt.legend(loc="upper left", bbox_to_anchor=(1, 1))  # Mettre la légende en dehors du graphe

# Afficher le graphique
plt.xticks(rotation=45)  # Rotation des années pour une meilleure lisibilité
plt.grid(True)
plt.show()